# Computing Bounds on Posterior Predictive Decision Probability with DeepBayes

Posterior Predictive: $p(y^* | x^*, X, Y) = \int p(y^{*} | x^{*}, \theta)p(\theta | X, Y) d\theta$

Here we walk through the code to compute upper and lower bounds on $p(y^* | x^*, X, Y)$ 

#### Example notebook takes 3 minutes to run in total
(Times are reported for an M1 Pro Macbook)

In [1]:
import os
import sys
import logging
import deepbayes
from deepbayes import PosteriorModel
from deepbayes.analyzers import decision_veri
from deepbayes.analyzers import FGSM

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#### Load in MNIST Dataset

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float64").reshape(-1, 28*28)
X_test = X_test.astype("float64").reshape(-1, 28* 28)

#### We define predicate for safety and worst-case logit values

The logit value defines the worst-case logit acheivable for a given input set and 
weight set. The lower bound proceedure then combines all of this information with 
the posterior probabilities of the weight sets to get a valid lower bound.

In [3]:
def predicate_safe(iml, imu, ol, ou):
    return True
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    if(np.argmax(worst_case) == TRUE_VALUE):
        return True
    else:
        return False
def logit_value(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    worst_case = tf.nn.softmax(worst_case)
    return worst_case[TRUE_VALUE]

#### Load in our pre-trained BNN posterior

In [4]:
bayes_model = PosteriorModel("PosteriorModels/VOGN_MNIST_Posterior/")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            100480    
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


#### Select an image that we would like to verify the robustness of

In [5]:
INDEX = 1
img = np.asarray([X_test[INDEX]])
TRUE_VALUE = np.argmax(bayes_model.predict(np.asarray([img]))) #y_test[INDEX]

#### Set Verification Parameters and compute Decision Lower Bound

##### Parameters: 
* Margin - The number of standard deviations that each weight sample will span
* Samples - The number of samples taken from the posterior (Small here for time savings)
* Max Depth - The depth of the Bonferroni Bound used to compute the probability
* Epsilon - The size of the input set that we consider for verification

In [6]:
MARGIN = 3.5
SAMPLES = 3
MAXDEPTH = 3
EPSILON = 0.01
img = np.asarray([X_test[INDEX]])
img_upper = np.clip(np.asarray([X_test[INDEX]+(EPSILON)]), 0, 1)
img_lower = np.clip(np.asarray([X_test[INDEX]-(EPSILON)]), 0, 1)
p_lower = decision_veri(bayes_model, img_lower, img_upper, MARGIN, SAMPLES, predicate=predicate_safe, value=logit_value, depth=MAXDEPTH)
print("Lowerbound on Decision Probability: ", p_lower)

Checking Samples:   0%|          | 0/3 [00:00<?, ?it/s]

--  tf.Tensor(0.98579246, shape=(), dtype=float32)
--  tf.Tensor(0.9856289, shape=(), dtype=float32)
--  tf.Tensor(0.98577034, shape=(), dtype=float32)


Checking Samples: 100%|██████████| 3/3 [00:00<00:00, 105.30it/s]


Found 3 safe intervals
About to compute intersection for this many intervals:  3
GOT THIS MANY VALUES:  3 [0.98579246 0.9856289  0.98577034]


Computing intersection weights:   0%|          | 0/3 [00:00<?, ?it/s]/Users/matthewwicker/AdversarialRobustnessOfBNNs/deepbayes/analyzers/probverification.py:496: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  stage1_args.append((model.posterior_mean, model.posterior_var, np.swapaxes(np.asarray([weight_intervals[wi]]),1,0), margin, verbose, n_proc, False))
Computing intersection weights: 100%|██████████| 3/3 [00:00<00:00, 5282.50it/s]

...

[0.9203388856672858, 0.9256142326759657, 0.9078951361373474] [0.98579246 0.9856289  0.98577034]
Depth 1 prob:  2.753848254480599 logit val:  2.7145513743526486
Depth 2 has 3 intersections


100%|██████████| 3/3 [00:29<00:00,  9.89s/it]


Current prob:  0.2255570033916241  Current dec:  0.22219976891232074
Depth 3 has 1 intersections


100%|██████████| 1/1 [00:27<00:00, 27.95s/it]


Current prob:  0.9994163431619119  Current dec:  0.9850473570534515
Got this approximation:  0.9994163431619119
Lowerbound on Decision Probability:  0.9844724274276666


In [7]:
def predicate_safe(iml, imu, ol, ou):
    return True

def logit_value(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    #best_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    #best_case = tf.nn.softmax(best_case)
    best_case = tf.math.add(tf.math.multiply(v1, ou), tf.math.multiply(v2, ol))
    best_case = tf.nn.softmax(best_case)
    return best_case[TRUE_VALUE]

#### Set Verification Parameters and compute Decision Upper Bound

##### Parameters: 
* Margin - The number of standard deviations that each weight sample will span
* Samples - The number of samples taken from the posterior (Small here for time savings)
* Max Depth - The depth of the Bonferroni Bound used to compute the probability
* Epsilon - The size of the input set that we consider for verification

In [8]:
from deepbayes.analyzers import decision_veri_upper
INDEX = 1
EPSILON = 0.15
MARGIN = 3.0
img = np.asarray([X_test[INDEX]])
img_upper = np.clip(np.asarray([X_test[INDEX]+(EPSILON)]), 0, 1)
img_lower = np.clip(np.asarray([X_test[INDEX]-(EPSILON)]), 0, 1)
p_upper = decision_veri_upper(bayes_model, img_lower, img_upper, MARGIN, SAMPLES, predicate=predicate_safe, value=logit_value, depth=MAXDEPTH)
print("Upperbound on Decision Probability: ", p_upper)

Checking Samples: 100%|██████████| 3/3 [00:04<00:00,  1.48s/it]


Found 3 safe intervals
About to compute intersection for this many intervals:  3
GOT THIS MANY VALUES:  3 [<tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]


Computing intersection weights: 100%|██████████| 3/3 [00:00<00:00, 50131.12it/s]

Depth 1 has 3 intersections



100%|██████████| 3/3 [00:25<00:00,  8.57s/it]


[0.1266824459031524, 0.09092414159744076, 0.10592504729348824] [<tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]
Depth 1 prob:  0.32353163479408137 logit val:  0.32353163479408137
Depth 2 has 3 intersections


100%|██████████| 3/3 [00:29<00:00, 10.00s/it]


Current prob:  0.28812130687441734  Current dec:  0.28812130687441734
Depth 3 has 1 intersections


100%|██████████| 1/1 [00:27<00:00, 27.71s/it]


Current prob:  0.2894305255215826  Current dec:  0.2894305255215826
Got this approximation:  0.2894305255215826
Upperbound on Decision Probability:  0.7943395035821169
